In [ ]:
import nltk

#downloading vader - only the first time
nltk.download('vader_lexicon')

[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


True

In [ ]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

sid = SentimentIntensityAnalyzer()

In [ ]:
import numpy as np
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/copy_eng_headlines_60.csv')
df.drop(columns=['blob_sentiment'], inplace=True)
df.head()

,Date,Title,sentiment
0,2017-01-05,Eliminating shadow economy to have positive im...,POSITIVE
1,2017-01-05,Two Chinese companies hit roadblock with India...,NEGATIVE
2,2017-01-05,SoftBank India Vision gets new $100,POSITIVE
3,2017-01-05,Nissan halts joint development of luxury cars ...,NEGATIVE
4,2017-01-05,Despite challenges Rajasthan continues to prog...,POSITIVE


In [ ]:
df['sentiment'].value_counts()

NEGATIVE    47835
POSITIVE    41828
Name: sentiment, dtype: int64

In [ ]:
import spacy

nlp = spacy.load('en_core_web_sm')

def tokenizer(text):
  token = [token.lemma_ for token in nlp(text)]
  return ' '.join(token)

In [ ]:
df['preprocessed_text'] = df['Title'].apply(tokenizer)

# Preview the preprocessed data
print(df[['Title', 'preprocessed_text']].head())

                                               Title  \
0  Eliminating shadow economy to have positive im...   
1  Two Chinese companies hit roadblock with India...   
2                SoftBank India Vision gets new $100   
3  Nissan halts joint development of luxury cars ...   
4  Despite challenges Rajasthan continues to prog...   

                                   preprocessed_text  
0  eliminate shadow economy to have positive impa...  
1  two chinese company hit roadblock with indian ...  
2                SoftBank India Vision get new $ 100  
3  Nissan halt joint development of luxury car wi...  
4  despite challenge Rajasthan continue to progre...  


In [ ]:
df['scores'] = df['preprocessed_text'].apply(lambda Title: sid.polarity_scores(Title))

df.head()

,Date,Title,sentiment,preprocessed_text,scores
0,2017-01-05,Eliminating shadow economy to have positive im...,POSITIVE,eliminate shadow economy to have positive impa...,"{'neg': 0.0, 'neu': 0.735, 'pos': 0.265, 'comp..."
1,2017-01-05,Two Chinese companies hit roadblock with India...,NEGATIVE,two chinese company hit roadblock with indian ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
2,2017-01-05,SoftBank India Vision gets new $100,POSITIVE,SoftBank India Vision get new $ 100,"{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'comp..."
3,2017-01-05,Nissan halts joint development of luxury cars ...,NEGATIVE,Nissan halt joint development of luxury car wi...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound..."
4,2017-01-05,Despite challenges Rajasthan continues to prog...,POSITIVE,despite challenge Rajasthan continue to progre...,"{'neg': 0.122, 'neu': 0.599, 'pos': 0.279, 'co..."


In [ ]:
df['compound'] = df['scores'].apply(lambda score_dict: score_dict['compound'])

In [ ]:
df.head()

,Date,Title,sentiment,preprocessed_text,scores,compound
0,2017-01-05,Eliminating shadow economy to have positive im...,POSITIVE,eliminate shadow economy to have positive impa...,"{'neg': 0.0, 'neu': 0.735, 'pos': 0.265, 'comp...",0.5574
1,2017-01-05,Two Chinese companies hit roadblock with India...,NEGATIVE,two chinese company hit roadblock with indian ...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
2,2017-01-05,SoftBank India Vision gets new $100,POSITIVE,SoftBank India Vision get new $ 100,"{'neg': 0.0, 'neu': 0.714, 'pos': 0.286, 'comp...",0.2500
3,2017-01-05,Nissan halts joint development of luxury cars ...,NEGATIVE,Nissan halt joint development of luxury car wi...,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000
4,2017-01-05,Despite challenges Rajasthan continues to prog...,POSITIVE,despite challenge Rajasthan continue to progre...,"{'neg': 0.122, 'neu': 0.599, 'pos': 0.279, 'co...",0.3773


In [ ]:
df['comp_sentiment'] = df['compound'].apply(lambda c: 'POSITIVE' if c > 0 else 'NEGATIVE')
#can change condition for pos from >= 0 to > 0.
df.head()

In [ ]:
df['comp_sentiment'].value_counts()

NEGATIVE    55361
POSITIVE    34302
Name: comp_sentiment, dtype: int64

In [ ]:
# let's now see the accuracy of VADER

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

In [ ]:
accuracy_score(df['sentiment'], df['comp_sentiment'])

0.6109878099104424

In [ ]:
print(classification_report(df['sentiment'], df['comp_sentiment']))

              precision    recall  f1-score   support

    NEGATIVE       0.62      0.71      0.66     47835
    POSITIVE       0.60      0.49      0.54     41828

    accuracy                           0.61     89663
   macro avg       0.61      0.60      0.60     89663
weighted avg       0.61      0.61      0.61     89663



In [ ]:
print(confusion_matrix(df['sentiment'], df['comp_sentiment']))

[[34158 13677]
 [21203 20625]]
